# Lab 4 - Linear models

We focus on models in the form 

$$ y ~ \mathrm{Normal}(\alpha+X\beta,\sigma) $$

or in generalized form (generalized linear models)

$$ f(y) ~ \mathrm{Normal}(\alpha+X\beta,\sigma) $$

where $f(y)$ is a link function, for example - logit.


In [1]:
from cmdstanpy import CmdStanModel
import arviz as az
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd

KeyboardInterrupt: 

## Excercise 1 - modelling height of !Kung people

### Normal model - no predictors
We will try to fit $\mathrm{Normal}(\mu,\sigma)$ distribution to height data. Special case of linear model with $\beta=0$.

In [ ]:
_BASE_URL = "https://raw.githubusercontent.com/rmcelreath/rethinking/Experimental/data"
HOWELL_DATASET_PATH = f"{_BASE_URL}/Howell1.csv"
d = pd.read_csv(HOWELL_DATASET_PATH, sep=";", header=0)
d = d[d.age >= 18]  # just adults
d.head()

In [ ]:
model_ppc = CmdStanModel(stan_file="Data/height_1_ppc.stan")

R = 1000
sim = model_ppc.sample(
    iter_sampling=R, iter_warmup=0, chains=1, fixed_param=True, seed=29042020, refresh=R
)

#### Task 1. Prior predictive checks

1. Plot histograms of mu, sigma and simulated height.
2. Plot a joint distribution of mu and sigma.
3. Check if samples are consistent with priors
4. Check if observed data is possible to obtain using priors.

In [ ]:
model_1_fit = CmdStanModel(stan_file="Data/height_1_fit.stan")

In [ ]:
fit = model_1_fit.sample(data=dict(N=len(d), heights=d.height.values), seed=28052020)

In [ ]:
fig, axs = plt.subplots(3, 1, sharey=True, tight_layout=True)
fig.set_size_inches(25, 12)
axs[0].hist(fit.stan_variable("mu"))
axs[0].set_title("Mu")
axs[1].hist(fit.stan_variable("sigma"))
axs[1].set_title("Sigma")
axs[2].hist(fit.stan_variable("height"))
axs[2].set_title("Simulated height")
plt.show()

In [ ]:
az.plot_pair(fit, var_names=["mu", "sigma"], figsize=(20, 6))

#### Task 2. Model fit and evaluation

1. Plot a joint distribution of fitted mu and sigma.
2. Plot histograms of data and simulated heights and evaluate the quality of model.



In [ ]:
fig, axs = plt.subplots(3, 1, sharey=True, tight_layout=True)
fig.set_size_inches(25, 12)
axs[0].hist(fit.stan_variable("mu"))
axs[0].set_title("Mu")
axs[1].hist(fit.stan_variable("sigma"))
axs[1].set_title("Sigma")
axs[2].hist(fit.stan_variable("height"))
axs[2].set_title("Simulated height")
plt.show()
az.plot_pair(fit, var_names=["mu", "sigma"], figsize=(20, 6))

### Adding predictor to the model - weight

Create column ```c_weight``` in the dataframe containing weights substrated by their mean.


In [ ]:
model_ppc = CmdStanModel(stan_file="Data/height_2a_ppc.stan")
R = 1000
weights = list(d["weight"])
mean = sum(weights) / len(weights)
c_weight = [elem - mean for elem in weights]
d["c_weight"] = c_weight

data_sim = {"N": 50, "weight": np.linspace(d.c_weight.min(), d.c_weight.max())}
sim = model_ppc.sample(
    data=data_sim,
    iter_sampling=R,
    iter_warmup=0,
    chains=1,
    refresh=R,
    fixed_param=True,
    seed=29042020,
)

#### Task 4. Prior predictive checks
1. Plot lines for each sampled slope beta and intercept alpha, verify if possible predicted heights are consistent with minimum (0) and maximum (check Wikipedia) heights observed in nature.

In [ ]:
fig, axs = plt.subplots(3, 1, tight_layout=True)
fig.set_size_inches(25, 20)
axs[0].plot(sim.stan_variable("alpha"))
axs[0].set_title("Alpha")
axs[1].plot(sim.stan_variable("beta"))
axs[1].set_title("Beta")
axs[2].plot(sim.stan_variable("height"))
axs[2].set_title("Heights")
height = sim.stan_variable("height")
print(
    "Minimum height: {0} \nMaximum height:  {1}".format(np.min(height), np.max(height))
)

### Modifying prior

If prior for beta admits negative values, then it makes no sense. Lets change prior to lognormal distribution.


In [ ]:
model_ppc = CmdStanModel(stan_file="Data/height_2b_ppc.stan")

In [ ]:
sim = model_ppc.sample(
    data=data_sim,
    iter_sampling=R,
    iter_warmup=0,
    chains=1,
    refresh=R,
    fixed_param=True,
    seed=29042020,
)

#### Task 5. Prior predictive checks
1. Plot lines for each sampled slope beta and intercept alpha, verify if possible predicted heights are consistent with minimum (0) and maximum (check Wikipedia) heights observed in nature.
2. For each simulated weight plot maximum, minimum, and 5, 25, 50, 75, 95 quantiles of simulated weight (all in the same plot). Compare with observed data. Is observed data possible within the prior model?

### Fitting data


In [ ]:
model_2_fit = CmdStanModel(stan_file="Data/height_2_fit.stan")

#### Task 6. Preparing data for fit
1. Create ```data_fit``` dictionary containing data from  ```N``` first rows of dataframe

In [ ]:
def sample_by_n(N: int):
    data_fit = {"N": N}
    data_fit["weight"] = list(d["weight"][:N])
    data_fit["heights"] = list(d["height"][:N])

    fit = model_2_fit.sample(data=data_fit, seed=28052020)
    return fit

#### Task 7. Evaluating model

1. Plot lines for each sampled slope beta and intercept alpha. Verify how uncertainity changes with increasing of sample (N)
2. For each simulated weight plot maximum, minimum, and 5, 25, 50, 75, 95 quantiles of simulated weight (all in the same plot). Compare with observed data (N points). Is observed data possible within the posterior model? What changes when N increases.


In [ ]:
fit = sample_by_n(100)

In [ ]:
summary = fit.summary()
height = fit.stan_variable("height")
print(
    "Minimum height: {0} \nMaximum height:  {1}".format(np.min(height), np.max(height))
)  
minimum = []
maximum = []
quanitile_5 = []
quanitile_25 = []
quanitile_50 = []
quanitile_75 = []
quanitile_95 = []
for i in range(len(height[0])):
    minimum.append(np.min(height[:,i]))
    maximum.append(np.max(height[:,i]))
    quanitile_5.append(np.quantile(height[:,i],0.05))
    quanitile_25.append(np.quantile(height[:,i],0.25))
    quanitile_50.append(np.quantile(height[:,i],0.5))
    quanitile_75.append(np.quantile(height[:,i],0.75))
    quanitile_95.append(np.quantile(height[:,i],0.95))

plt.rcParams["figure.figsize"] = (20, 8)
plt.plot(minimum, linewidth=2.5)
plt.plot(maximum, linewidth=2.5)
plt.plot(quanitile_5, linewidth=0.5)
plt.plot(quanitile_25, linewidth=0.5)
plt.plot(quanitile_50, linewidth=0.5)
plt.plot(quanitile_75, linewidth=0.5)
plt.plot(quanitile_95, linewidth=0.5)
plt.legend(['min','max','5','25','50','75','95'])
plt.title('Model evaluation')
plt.show()